# Gan Losses

In [ ]:
import numpy as np

In [ ]:
a = np.arange(0.1,0.9,0.1)

In [ ]:
def sigmoid(x):
    return 1/(1+np.exp(-x))
def lossg(x):
    return np.log(1-sigmoid(x))
def lossgnew(x):
    return -x+np.log(sigmoid(x))

In [ ]:
lossg(a)-lossgnew(a)

# Split data and attenuation kernel

In [ ]:
import numpy as np
from gantools.data.Dataset import slice_2d_patch, slice_3d_patch
from gantools.plot import draw_images
from gantools import plot
%matplotlib inline
import tensorflow as tf
from matplotlib import pyplot as plt
import os
os.environ["CUDA_VISIBLE_DEVICES"]=""

from gantools import blocks

from gantools.data.transformation import slice_3d_patch, slice_2d_patch

In [ ]:
def attenuation_kernel(nx=32):
    x = np.arange(nx,0,-1)/nx
    x -= 0.5
    x[x<0] = 0
    return 2*x

def attenuation_weight_2d(nx=32):
    k = attenuation_kernel(nx)
    kk = np.concatenate((np.ones([nx]), k))[::-1]
    kk = np.expand_dims(kk, axis=0)
    aw = np.ones([nx*2,nx*2])
    aw = kk.T*aw*kk
    return aw

def attenuation_weight_3d(nx=32):
    k = attenuation_kernel(nx)
    kk = np.concatenate((np.ones([nx]), k))[::-1]
    kk = np.expand_dims(np.expand_dims(kk, axis=0), axis=0)
    aw = np.ones([nx*2,nx*2,nx*2])
    aw = (aw*kk)*kk.transpose((1,2,0))*kk.transpose((2,0,1))
    return aw

In [ ]:
nx= 32
k = attenuation_kernel(nx)
kk = np.concatenate((np.ones([nx]), k))[::-1]
plt.plot(kk)

In [ ]:
nx = 32
aw = attenuation_weight_2d(nx)
plt.imshow(aw)
slice_2d_patch(np.expand_dims(aw, axis=0), spix=nx)[-1:,:,:,:3].shape

In [ ]:
aw3 = attenuation_weight_3d(nx)
plt.imshow(plot.tile_cube_to_2d(aw3))
slice_3d_patch(np.expand_dims(aw3, axis=0))[-1:,:,:,:,:7].shape

In [ ]:
def path2img(patches, is_3d=False):
    if is_3d:
        imgs_down_left = np.concatenate([patches[:, :, :, :, 3], patches[:, :, :, :,2]], axis=2)
        imgs_down_right = np.concatenate([patches[:, :, :, :, 1], patches[:, :, :, :,0]], axis=2)
        imgs_down = np.concatenate([imgs_down_left, imgs_down_right], axis=3)
        imgs_up_left   = np.concatenate([patches[:, :, :, :, 7], patches[:, :, :, :, 6]], axis=2)
        imgs_up_right  = np.concatenate([patches[:, :, :, :, 5], patches[:, :, :, :, 4]], axis=2)
        imgs_up = np.concatenate([ imgs_up_left, imgs_up_right], axis=3)
        imgs = np.concatenate([imgs_up, imgs_down], axis=1)
    else:
        imgs_d = np.concatenate(
            [patches[:, :, :, 1], patches[:, :, :, 0]], axis=1)
        imgs_u = np.concatenate(
            [patches[:, :, :, 3], patches[:, :, :, 2]], axis=1)
        imgs = np.concatenate([imgs_u, imgs_d], axis=2)
    return imgs

def tf_patch2img_2d(dr, dl, ur, ul):

    imgs_d = tf.concat([dl, dr], axis=1)
    imgs_u = tf.concat([ul, ur], axis=1)
    imgs = tf.concat([imgs_u,  imgs_d], axis=2)
    return imgs

def tf_patch2img_3d(*args):
    imgs_down_left = tf.concat([args[3], args[2]], axis=2)
    imgs_down_right = tf.concat([args[1], args[0]], axis=2)
    imgs_down = tf.concat([imgs_down_left, imgs_down_right], axis=3)
    imgs_up_left   = tf.concat([args[7], args[6]], axis=2)
    imgs_up_right  = tf.concat([args[5], args[4]], axis=2)
    imgs_up = tf.concat([ imgs_up_left, imgs_up_right], axis=3)
    imgs = tf.concat([imgs_up, imgs_down], axis=1)
    return imgs

def flip_slices_2d(dl, ur, ul):
    flip_dl = np.flip(dl, axis=1)
    flip_ur = np.flip(ur, axis=2)    
    flip_ul = np.flip(np.flip(ul, axis=1), axis=2)
    return flip_dl, flip_ur, flip_ul

def tf_flip_slices_2d(dl, ur, ul):
    flip_dl = tf.reverse(dl, axis=[1])
    flip_ur = tf.reverse(ur, axis=[2])    
    flip_ul = tf.reverse(ul, axis=[1,2])
    return flip_dl, flip_ur, flip_ul


def flip_slices_3d(*args):
    flip_d_above = np.flip(args[0], axis=2)
    flip_d_left = np.flip(args[1], axis=3)
    flip_d_corner = np.flip(np.flip(args[2], axis=2), axis=3)
    flip_up = np.flip(args[3], axis=1)
    flip_u_above = np.flip(np.flip(args[4], axis=1), axis=2)
    flip_u_left = np.flip(np.flip(args[5], axis=1), axis=3)
    flip_u_corner = np.flip(np.flip(np.flip(args[6], axis=1), axis=2), axis=3)
    return flip_d_above, flip_d_left, flip_d_corner, flip_up, flip_u_above, flip_u_left, flip_u_corner

def tf_flip_slices_3d(*args):
    flip_d_above = tf.reverse(args[0], axis=[2])
    flip_d_left = tf.reverse(args[1], axis=[3])
    flip_d_corner = tf.reverse(args[2], axis=[2, 3])
    flip_up = tf.reverse(args[3], axis=[1])
    flip_u_above = tf.reverse(args[4], axis=[1, 2])
    flip_u_left = tf.reverse(args[5], axis=[1, 3])
    flip_u_corner = tf.reverse(args[6], axis=[1, 2, 3])
    return flip_d_above, flip_d_left, flip_d_corner, flip_up, flip_u_above, flip_u_left, flip_u_corner

In [ ]:
def test_patch2img_2d():
    imgs = np.random.rand(25,64,64)
    patches = slice_2d_patch(imgs,spix=32)
    imgs2 = path2img(patches, is_3d=False)
    np.testing.assert_almost_equal(imgs2[-25:],imgs)

def test_patch2img_3d():
    imgs = np.random.rand(25,64,64,64)
    patches = slice_3d_patch(imgs,spix=32)
    imgs2 = path2img(patches, is_3d=True)
    np.testing.assert_almost_equal(imgs2[-25:],imgs)

    
def test_tf_patch2img_2d():
    imgs = np.random.rand(25,64,64)
    patches = slice_2d_patch(imgs,spix=32)
    imgs2 = path2img(patches, is_3d=False)
    args = (patches[:,:,:,0], patches[:,:,:,1], patches[:,:,:,2], patches[:,:,:,3])
    with tf.Session() as sess:
        imgs3 = tf_patch2img_2d(*args).eval()
    np.testing.assert_almost_equal(imgs3,imgs2)

def test_tf_patch2img_3d():
    imgs = np.random.rand(25,16,16,16)
    patches = slice_3d_patch(imgs,spix=8)
    imgs2 = path2img(patches, is_3d=True)
    args = (patches[:,:,:,:,0], patches[:,:,:,:,1], patches[:,:,:,:,2], patches[:,:,:,:,3],
            patches[:,:,:,:,4], patches[:,:,:,:,5], patches[:,:,:,:,6], patches[:,:,:,:,7])
    with tf.Session() as sess:
        imgs3 = tf_patch2img_3d(*args).eval()
    np.testing.assert_almost_equal(imgs3,imgs2)    

def test_flip_slices_2d():
    imgs = np.zeros([25,16,16])
    imgs[:,4:12,4:12] = 1
    patches = slice_2d_patch(imgs,spix=8)[-25:]
    dr, dl, ur, ul = patches[:,:,:,0:1], patches[:,:,:,1:2], patches[:,:,:,2:3], patches[:,:,:,3:4]
    flip_dl, flip_ur, flip_ul = flip_slices_2d(dl, ur, ul)
    np.testing.assert_almost_equal(dr,flip_dl)
    np.testing.assert_almost_equal(dr,flip_ur)
    np.testing.assert_almost_equal(dr,flip_ul)

def test_flip_slices_3d():
    imgs = np.zeros([25,16,16,16])
    imgs[:,4:12,4:12,4:12] = 1
    patches = slice_3d_patch(imgs,spix=8)[-25:]
    args = (patches[:,:,:,:,0:1], patches[:,:,:,:,1:2], patches[:,:,:,:,2:3], patches[:,:,:,:,3:4],
            patches[:,:,:,:,4:5], patches[:,:,:,:,5:6], patches[:,:,:,:,6:7], patches[:,:,:,:,7:8])
    sols = flip_slices_3d(*args[1:])
    for sol in sols:
        np.testing.assert_almost_equal(args[0], sol)
    
def test_tf_flip_slice_2d():
    imgs = np.random.rand(25,64,64)
    patches = slice_2d_patch(imgs,spix=32)
    args = (patches[:,:,:,1], patches[:,:,:,2], patches[:,:,:,3])
    a,b,c = flip_slices_2d(*args)

    with tf.Session() as sess:
        a2t,b2t,c2t = tf_flip_slices_2d(*args)
        a2, b2, c2 = a2t.eval(), b2t.eval(), c2t.eval()
    np.testing.assert_almost_equal(a,a2)
    np.testing.assert_almost_equal(b,b2)
    np.testing.assert_almost_equal(c,c2)
    

def test_tf_flip_slice_3d():
    imgs = np.random.rand(25,16,16,16)
    patches = slice_3d_patch(imgs,spix=8)
    args = (patches[:,:,:,:,1], patches[:,:,:,:,2], patches[:,:,:,:,3],
            patches[:,:,:,:,4], patches[:,:,:,:,5], patches[:,:,:,:,6], patches[:,:,:,:,7])
    s1s = flip_slices_3d(*args)

    with tf.Session() as sess:
        tfst = tf_flip_slices_3d(*args)
        s2s = []
        for st in tfst:
            s2s.append(st.eval())
    for s1,s2 in zip(s1s,s2s): 
        np.testing.assert_almost_equal(s1,s2)
        
test_patch2img_2d()
test_tf_patch2img_2d()
test_flip_slices_2d()
test_tf_flip_slice_2d()

test_patch2img_3d()
test_tf_patch2img_3d()
test_flip_slices_3d()
test_tf_flip_slice_3d()

# Test convolution

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import numpy as np
from  scipy.signal import convolve2d
from gantools import blocks
import tensorflow as tf
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
from matplotlib import pyplot as plt

In [ ]:
X = np.random.randn(*[6,7,8])
K = np.random.randn(6,3,5)

In [ ]:
# Scipy
s = np.zeros(X.shape[1:])
for x,k in zip(X,K):
#     s.append(convolve2d(x, k, mode='same', boundary='symm'))
    s+=convolve2d(x, k[::-1,::-1], mode='same', boundary='fill',fillvalue=0)
# Tensorflow
tfX = tf.constant(np.expand_dims(X.transpose((1,2,0)),0))
tfk = tf.constant(np.expand_dims(K.transpose((1,2,0)),3))
stride = [1,1]
# imgs = tf.pad(imgs,[])
conv = tf.nn.conv2d(tfX, tfk, strides=[1, *stride, 1], padding='SAME')
with tf.Session() as sess:
    convtf = conv.eval().reshape(X.shape[1:])
assert(np.sum(np.abs(s-convtf))<1e-12)

In [ ]:
# Scipy
s = np.zeros(X.shape[1:])
for x,k in zip(X,K):
#     s.append(convolve2d(x, k, mode='same', boundary='symm'))
    s+=convolve2d(x, k[::-1,::-1], mode='same', boundary='fill')
# Tensorflow
tfX = tf.constant(np.expand_dims(X.transpose((1,2,0)),0))
tfk = tf.constant(np.expand_dims(K.transpose((1,2,0)),3))
stride = [1,1]
tfX = tf.pad(tfX,[[0,0],[1,1],[2,2],[0,0]], mode='CONSTANT')
conv = tf.nn.conv2d(tfX, tfk, strides=[1, *stride, 1], padding='VALID')
with tf.Session() as sess:
    convtf = conv.eval().reshape(X.shape[1:])
assert(np.sum(np.abs(s-convtf))<1e-12)

In [ ]:
# Scipy
s = np.zeros(X.shape[1:])
for x,k in zip(X,K):
#     s.append(convolve2d(x, k, mode='same', boundary='symm'))
    s+=convolve2d(x, k[::-1,::-1], mode='same', boundary='symm')
# Tensorflow
tfX = tf.constant(np.expand_dims(X.transpose((1,2,0)),0))
tfk = tf.constant(np.expand_dims(K.transpose((1,2,0)),3))
stride = [1,1]
tfX = tf.pad(tfX,[[0,0],[1,1],[2,2],[0,0]], mode='SYMMETRIC')
conv = tf.nn.conv2d(tfX, tfk, strides=[1, *stride, 1], padding='VALID')
with tf.Session() as sess:
    convtf = conv.eval().reshape(X.shape[1:])
assert(np.sum(np.abs(s-convtf))<1e-12)

In [ ]:
X = np.ones([1,7,8])
K = np.ones([1,7,6])
tfX = tf.constant(np.expand_dims(X.transpose((1,2,0)),0))
tfk = tf.constant(np.expand_dims(K.transpose((1,2,0)),3))
stride = [1,1]
# tfX = tf.pad(tfX,[[0,0],[1,1],[2,2],[0,0]], mode='CONSTANT')
conv = tf.nn.conv2d(tfX, tfk, strides=[1, *stride, 1], padding='SAME')
with tf.Session() as sess:
    convtf = conv.eval().reshape(X.shape[1:])

K11 = (tfk.shape[0]-1)//2
K12 = tfk.shape[0]//2
K21 = (tfk.shape[1]-1)//2
K22 = tfk.shape[1]//2

tfX = tf.pad(tfX,[[0,0],[K11,K12],[K21,K22],[0,0]], mode='CONSTANT')
conv = tf.nn.conv2d(tfX, tfk, strides=[1, *stride, 1], padding='VALID')
with tf.Session() as sess:
    convtf2 = conv.eval().reshape(X.shape[1:])
    
assert(np.sum(np.abs(convtf2-convtf))<1e-12)

# FFT/PSD tests


In [ ]:
import numpy as np
import tensorflow as tf
import scipy

In [ ]:
s = np.ones([4,4])
ns = 32
for ns in range(1,35,2):
    d = (np.arange(ns) - ns//2)**2
    d = np.expand_dims(d,axis=0)
    DD = d.T+d
    dd = DD.flatten()
    print(len(np.unique(dd)))

In [ ]:
d = (np.arange(ns) - ns//2)**2
d = np.expand_dims(d,axis=0)
DD = np.fft.ifftshift(d.T+d).flatten()
dd = np.unique(DD)
mat = np.zeros(shape=(len(dd), len(DD)))
for it,v in enumerate(dd):
    mat[it] = (DD==v)

In [ ]:
plt.imshow(mat)

In [ ]:
scipy.sparse.coo_matrix(mat)

In [ ]:
for ns in range(1,35,2):
    d = (np.arange(ns) - ns//2)**2
    d = np.expand_dims(d,axis=0)
    d = np.expand_dims(d,axis=0)
    DD = d.transpose((0,1,2))+d.transpose((1,2,0))+d.transpose((2,0,1))
    dd = DD.flatten()
    print(len(np.unique(dd)))

In [ ]:
d = (np.arange(ns) - ns//2)**2
d = np.expand_dims(d,axis=0)
d = np.expand_dims(d,axis=0)
DD = d.transpose((0,1,2))+d.transpose((1,2,0))+d.transpose((2,0,1))
DD = np.fft.ifftshift(DD).flatten()
dd = np.unique(DD)
mat = np.zeros(shape=(len(dd), len(DD)))
for it,v in enumerate(dd):
    mat[it] = (DD==v)

In [ ]:
plt.imshow(mat)

In [ ]:
scipy.sparse.coo_matrix(mat)

In [ ]:
def get_fourier_sum_matrix(ns, dim):
    d = (np.arange(ns) - ns//2)**2
    if dim==2:
        d = np.expand_dims(d,axis=0)
        DD = np.fft.ifftshift(d.T+d).flatten()
        dd = np.unique(DD)        
    elif dim==3:
        d = np.expand_dims(d,axis=0)
        d = np.expand_dims(d,axis=0)
        DD = d.transpose((0,1,2))+d.transpose((1,2,0))+d.transpose((2,0,1))
        DD = np.fft.ifftshift(DD).flatten()
        dd = np.unique(DD)
    else:
        raise ValueError()
    mat = np.zeros(shape=(len(dd), len(DD)))
    for it,v in enumerate(dd):
        mat[it] = (DD==v)
    return scipy.sparse.coo_matrix(mat)

In [ ]:
S = get_fourier_sum_matrix(64,2)
S = S.transpose().astype(np.float32)


In [ ]:
tfS = tf.SparseTensor(indices=np.array([S.row, S.col]).T,
            values=S.data,
            dense_shape=S.shape)

In [ ]:
with tf.Session() as sess:
    print(tf.shape(tfS).eval())

In [ ]:
from gantools.blocks import *
ns = 16
X = np.random.rand(10,ns,ns)
X = np.expand_dims(np.sin(2*np.pi*np.arange(ns)/ns), 0)
X = np.expand_dims(np.sin(2*np.pi*np.arange(ns)/ns), 2)
X = X*np.ones([1,1,ns])
print(X.shape)
# X = np.ones((10,ns,ns))

fftX = tf.abs(tf.fft2d(X))/tf.constant(ns, dtype=tf.float32)
fftX = tf.expand_dims(fftX,axis=2+1)

S = get_fourier_sum_matrix(ns,2).astype(np.float32)

print(S.shape)
tfS = tf.SparseTensor(
    indices=np.array([S.row, S.col]).T,
    values=S.data,
    dense_shape=S.shape)
fftx = reshape2d(fftX)
print(fftx.shape)
with tf.Session() as sess:
    print(fftX.eval()[0,:,:,0])
    
    print(tf.shape(tfS).eval())
    
# psd_features =  tf.matmul(fftx, tfS)
psd_features = tf.transpose(tf.sparse_tensor_dense_matmul(tfS, fftx, adjoint_a=False, adjoint_b=True))
psd_features.shape


In [ ]:
with tf.Session() as sess:
    print(psd_features.eval()[0])